In [101]:
import requests,json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



# Functions for getting SensorTower data. 
# Store_Intelligence
# Rate Limit: 6 requests per second. 
# Requests should be made to the following base URL, followed by the specific path to the API you want to use.
# Latest Updated Date: 2021.03.02
# Author: Chen

# Compact Download / Revenue Estimates ???


# Download / Revenue Estimates for applications (If run error please try https://api.sensortower-china.com:443)

# Input: 
# os: Operating System (use "ios", "android",or "unified")
# country: Checking CountryCodes_CategoryIDs to select country (use"WW" for worldwide, "US"  for United States)  
# date_granularity: use "daily", "weekly","monthly", or"quarterly",defaults to "daily"
# start_date:(yyyy-mm-dd) 
# end_date: (yyyy-mm-dd)  
# app_ids: (defaults to None) 
# publisher_ids: (defaults to None) 
# Do not give both app_ids and publisher_ids at same time. 

# Output: A dataframe which containz: 
 
# {
#   "sales_report_estimates_key": {
#     "ios": {
#       "aid": "App ID",
#       "cc": "Country Code",
#       "d": "Date",
#       "iu": "iPhone Downloads",
#       "ir": "iPhone Revenue",
#       "au": "iPad Downloads",
#       "ar": "iPad Revenue"
#     },
#     "android": {
#       "aid": "App ID",
#       "c": "Country Code",
#       "d": "Date",
#       "u": "Android Downloads",
#       "r": "Android Revenue"
#     },
#     "unified": {
#       "app_id": "App ID",
#       "country": "Country Code",
#       "date": "Date",
#       "android_units": "Android Downloads",
#       "android_revenue": "Android Revenue",
#       "ipad_units": "iPad Downloads",
#       "ipad_revenue": "iPad Revenue",
#       "iphone_units": "iPhone Downloads",
#       "iphone_revenue": "iPhone Revenue"
#     }
#   }
# }
# If used publisher_ids, all applications made by this publisher will return. 

# Example: 
# get_data_apps = requests_Download_RevenueEstimates('ios', 'US', 'daily', '2020-11-01', '2020-11-04', '1531465966')
# get_data_publishers = requests_Download_RevenueEstimates('ios', 'US', 'daily', '2020-11-01', '2020-11-04', publisher_ids = '1191495496') 

def requests_Download_RevenueEstimates(os, country, date_granularity, start_date, end_date, app_ids = None, publisher_ids = None):
    if app_ids == None and publisher_ids == None:
        return(print('Error, at least one app ID, or one publisher ID is required.'))
        
    elif publisher_ids == None:
        aux_DRE = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/sales_report_estimates?app_ids='+app_ids+
                               '&countries='+country+
                               '&date_granularity='+date_granularity+
                               '&start_date='+
                               start_date+'&end_date='+
                               end_date+'&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    else:
        aux_DRE = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/sales_report_estimates?publisher_ids='+publisher_ids+
                               '&countries='+country+
                               '&date_granularity='+date_granularity+
                               '&start_date='+
                               start_date+'&end_date='+
                               end_date+'&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_DRE = pd.DataFrame(aux_DRE.json())
    print(aux_DRE.status_code)
    return(df_DRE)


# Top Apps(If run error please try https://api.sensortower-china.com:443) for one day, week, month, quarter. 给出指定日期后，会寻找该日期所在的日，周，月，季度，
# Input:
# os: Operating System (use "ios", "android",or "unified")
# comparision_attribute: use "absolute", "delta", or "transformed_delta"
# time_range: use "day","week", "month", or "quarter"
# measure: use "units" or "revenue"
# device_type: use "iphone", "ipad", or "total" for `ios`, leave blank for `android`, use "total" for `unified`
# category: Checking CountryCodes_CategoryIDs to select category.(use 6014 for game applications)
# date: Date.YYYY-MM-DD format.Auto-changes to the beginning of time_range.Ex: Mondays for weeks, 1st of the month, 1st day of the quarter, 1st day of the year.
# country: Checking CountryCodes_CategoryIDs to select country (use"WW" for worldwide, "US"  for United States)
# limit: Limit how many apps per call. (Default: 25, Max: 2000)

# Output: A dataframe which contains informations of top Apps. 
# The variable custom_tags contains informations about each application. 

# Example_ios_uni: 
# https://api.sensortower-china.com/v1/ios/sales_report_estimates_comparison_attributes?comparison_attribute=absolute&time_range=week&measure=units&device_type=total&category=6014&date=2021-02-08&country=US&limit=25&auth_token=r7cTTJq5nBSqxMU_WmJo
# get_week_AppRanking1 = requests_ios_uni_TopApps_periodic('ios', 'absolute', 'week','units', 'total', '6014', '2021-02-08','US','25') # 2021-02-08---2021-02-14
# get_week_AppRanking2 = requests_ios_uni_TopApps_periodic('ios', 'absolute', 'week','units', 'total', '6014', '2021-02-07','US','25') # 2021-02-01---2021-02-7
# get_week_AppRanking3 = requests_ios_uni_TopApps_periodic('ios', 'absolute', 'week','units', 'total', '6014', '2021-02-03','US','25') # 2021-02-01---2021-02-7


# Example_Android: 
# https://api.sensortower-china.com/v1/android/sales_report_estimates_comparison_attributes?comparison_attribute=absolute&time_range=week&measure=units&category=game&date=2021-02-08&country=US&limit=25&auth_token=r7cTTJq5nBSqxMU_WmJo
# get_week_ranking4 = requests_Android_TopApps_periodic('android', 'absolute', 'week','units', 'game', '2021-02-08','US','25')

def requests_ios_uni_TopApps_periodic(os,comparision_attribute, time_range, measure, device_type, category, date, country, limit):  
    aux_TopApps = requests.get('https://api.sensortower-china.com:443/v1/'
                               +os+'/sales_report_estimates_comparison_attributes?comparison_attribute='+comparision_attribute+
                               '&time_range='+time_range+
                               '&measure='+measure+
                               '&device_type='+device_type+
                               '&category='+category+
                               '&date='+date+
                               '&country='+country+
                               '&limit='+limit+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_TopApps = pd.DataFrame(aux_TopApps.json())
    print(aux_TopApps.status_code) # Get response messages
    return(df_TopApps)


# def requests_TopApps_customized(os,comparision_attribute, measure, device_type, category, date, end_date, country, limit):  
#     aux_TopApps = requests.get('https://api.sensortower-china.com:443/v1/'
#                                +os+'/sales_report_estimates_comparison_attributes?comparison_attribute='+comparision_attribute+
#                                '&measure='+measure+
#                                '&device_type='+device_type+
#                                '&category='+category+
#                                '&date='+date+
#                                '&end_date='+end_date+
#                                '&country='+country+
#                                '&limit='+limit+
#                                '&auth_token=r7cTTJq5nBSqxMU_WmJo'
#                                )
#     df_TopApps = pd.DataFrame(aux_TopApps.json())
#     print(aux_TopApps.status_code) # Get response messages
#     return(df_TopApps)    


def requests_Android_TopApps_periodic(os,comparision_attribute, time_range, measure, category, date, country, limit):  
    aux_TopApps = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/sales_report_estimates_comparison_attributes?comparison_attribute='+comparision_attribute+
                               '&time_range='+time_range+
                               '&measure='+measure+
                               '&category='+category+
                               '&date='+date+
                               '&country='+country+
                               '&limit='+limit+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_TopApps = pd.DataFrame(aux_TopApps.json())
    print(aux_TopApps.status_code) # Get response messages
    return(df_TopApps)


# Top Publishers(If run error please try https://api.sensortower-china.com:443) for one day, week, month, quarter. 给出指定日期后，会寻找该日期所在的日，周，月，季度，
# Input:
# os: Operating System (use "ios", "android",or "unified")
# comparision_attribute: use "absolute", "delta", or "transformed_delta"
# time_range: use "day","week", "month", or "quarter"
# measure: use "units" or "revenue"
# device_type: use "iphone", "ipad", or "total" for `ios`, leave blank for `android` and `unified`
# category: Checking CountryCodes_CategoryIDs to select category.(use 6014 for game applications)
# date: Date.YYYY-MM-DD format.Auto-changes to the beginning of time_range.Ex: Mondays for weeks, 1st of the month, 1st day of the quarter, 1st day of the year.
# country: Checking CountryCodes_CategoryIDs to select country (use"WW" for worldwide, "US"  for United States)
# limit: Limit how many apps per call. (Default: 25, Max: 2000)

# Output: A dataframe which contains informations of top Publishers. 
# The variable apps contains all informations of applications made by each publisher. 

# Example_ios: 
# https://api.sensortower-china.com/v1/ios/top_and_trending/publishers?comparison_attribute=absolute&time_range=week&measure=units&device_type=total&category=6014&date=2021-02-08&country=US&limit=25&auth_token=r7cTTJq5nBSqxMU_WmJo
# get_week_PubRanking1_ios = requests_ios_TopPublishers_periodic('ios', 'absolute', 'week','units', 'total', '6014', '2021-02-08','US','25')

# Example_Android_uni: 
# https://api.sensortower-china.com/v1/android/top_and_trending/publishers?comparison_attribute=absolute&time_range=week&measure=units&category=6014&date=2021-02-08&country=US&limit=25&auth_token=r7cTTJq5nBSqxMU_WmJo
# get_week_PubRanking1_android = requests_Android_uni_TopPublishers_periodic('android', 'absolute', 'week','units', 'game', '2021-02-08','US','25')

def requests_ios_TopPublishers_periodic(os,comparision_attribute, time_range, measure, device_type, category, date, country, limit):  
    aux_TopPublishers = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/top_and_trending/publishers?comparison_attribute='+comparision_attribute+
                               '&time_range='+time_range+
                               '&measure='+measure+
                               '&device_type='+device_type+
                               '&category='+category+
                               '&date='+date+
                               '&country='+country+
                               '&limit='+limit+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_TopPublishers = pd.DataFrame(aux_TopPublishers.json())
    print(aux_TopPublishers.status_code) # Get response messages
    return(df_TopPublishers)


def requests_Android_uni_TopPublishers_periodic(os,comparision_attribute, time_range, measure, category, date, country, limit):  
    aux_TopPublishers = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/top_and_trending/publishers?comparison_attribute='+comparision_attribute+
                               '&time_range='+time_range+
                               '&measure='+measure+
                               '&category='+category+
                               '&date='+date+
                               '&country='+country+
                               '&limit='+limit+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_TopPublishers = pd.DataFrame(aux_TopPublishers.json())
    print(aux_TopPublishers.status_code) # Get response messages
    return(df_TopPublishers)



# Store Summary: Information for all ios or android.
# Input:
# Output: 


def requests_Store_Summary(os, category, country, granularity, start_date, end_date):  
    aux_SS = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/store_summary?categories='+category+
                               '&countries='+country+
                               '&date_granularity='+granularity+
                               '&start_date='+start_date+
                               '&end_date='+end_date+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_SS = pd.DataFrame(aux_SS.json())
    print(aux_SS.status_code) # Get response messages
    return(df_SS)

# Game Breakdown: Information for all ios or android.(games specific)
# Input:
# Output: 

def requests_Games_Breakdown(os, category, country, granularity, start_date, end_date):  
    aux_GB = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/games_breakdown?categories='+category+
                               '&countries='+country+
                               '&date_granularity='+granularity+
                               '&start_date='+start_date+
                               '&end_date='+end_date+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_GB = pd.DataFrame(aux_GB.json())
    print(aux_GB.status_code) # Get response messages
    return(df_GB)



In [113]:
get_week_PubRanking1_ios = requests_ios_TopPublishers_periodic('ios', 'absolute', 'week','units', 'total', '6014', '2021-02-08','US','25')
get_week_PubRanking1_ios

200


,apps,date,publisher_id,publisher_name,revenue_absolute,revenue_delta,revenue_transformed_delta,units_absolute,units_delta,units_transformed_delta
0,"[{'app_id': 1545306176, 'canonical_country': '...",2021-02-08T00:00:00Z,295913422,Zynga Inc.,424298925,3259273,0.007741,2363669,-87416,-0.035664
1,"[{'app_id': 1533442608, 'canonical_country': '...",2021-02-08T00:00:00Z,714804730,Voodoo,1652289,-54734,-0.032064,1947086,-86486,-0.042529
2,"[{'app_id': 1549356463, 'canonical_country': '...",2021-02-08T00:00:00Z,1362220666,Lion Studios,8587508,-1042265,-0.108234,1939710,69855,0.037359
3,"[{'app_id': 1539408727, 'canonical_country': '...",2021-02-08T00:00:00Z,721307559,Crazy Labs,2979941,221862,0.080441,931190,13436,0.014640
4,"[{'app_id': 1543160727, 'canonical_country': '...",2021-02-08T00:00:00Z,1405039362,SayGames LLC,4170155,479379,0.129886,911042,58197,0.068239
5,"[{'app_id': 1546855758, 'canonical_country': '...",2021-02-08T00:00:00Z,1191495496,Good Job Games,2030338,-58969,-0.028224,908727,-179450,-0.164909
6,"[{'app_id': 1515957122, 'canonical_country': '...",2021-02-08T00:00:00Z,1499845738,SUPERSONIC STUDIOS LTD,1033623,185566,0.218813,856275,28823,0.034833
7,"[{'app_id': 1443446174, 'canonical_country': '...",2021-02-08T00:00:00Z,1441389613,Amanotes Pte. Ltd.,19385137,-1081257,-0.052831,552580,35000,0.067622
8,"[{'app_id': 1536131108, 'canonical_country': '...",2021-02-08T00:00:00Z,1483575279,TapNation,796932,-66859,-0.077402,527849,-85479,-0.139369
9,"[{'app_id': 1535973778, 'canonical_country': '...",2021-02-08T00:00:00Z,1422558565,JoyPac,18249,-14670,-0.445639,474948,-366476,-0.435543


In [115]:
get_week_PubRanking1_android = requests_Android_uni_TopPublishers_periodic('android', 'absolute', 'week','units', 'game', '2021-02-08','US','25')
get_week_PubRanking1_android

200


,apps,date,publisher_id,publisher_name,revenue_absolute,revenue_delta,revenue_transformed_delta,units_absolute,units_delta,units_transformed_delta
0,"[{'app_id': 'com.tgame.girlgenius', 'canonical...",2021-02-08T00:00:00Z,Lion+Studios,Lion Studios,5861813,-977522,-0.142926,1531665,-9209,-0.005976
1,"[{'app_id': 'com.uncosoft.highheels', 'canonic...",2021-02-08T00:00:00Z,Zynga,Zynga,296249335,-19550034,-0.061907,1178207,-68260,-0.054763
2,"[{'app_id': 'com.pixelbox.roofrails', 'canonic...",2021-02-08T00:00:00Z,VOODOO,VOODOO,467594,-15288,-0.031660,1099472,-219564,-0.166458
3,"[{'app_id': 'com.sdpgames.sculptpeople', 'cano...",2021-02-08T00:00:00Z,Crazy+Labs+by+TabTale,Crazy Labs by TabTale,3006688,420196,0.162458,945379,-37844,-0.038490
4,"[{'app_id': 'com.playstrom.dop2', 'canonical_c...",2021-02-08T00:00:00Z,SayGames,SayGames,1836682,88354,0.050536,865816,55095,0.067958
5,"[{'app_id': 'com.yolostudio.theimpostor', 'can...",2021-02-08T00:00:00Z,OneSoft+Global+PTE.+LTD.,OneSoft Global PTE. LTD.,21659162,2224792,0.114477,856886,-4358,-0.005060
6,"[{'app_id': 'com.hit.master', 'canonical_count...",2021-02-08T00:00:00Z,CASUAL+AZUR+GAMES,CASUAL AZUR GAMES,1242040,-216164,-0.148240,795006,-61286,-0.071571
7,"[{'app_id': 'com.freeplay.runandfight', 'canon...",2021-02-08T00:00:00Z,Supersonic+Studios+LTD,Supersonic Studios LTD,170945,-18038,-0.095448,661585,-564,-0.000852
8,"[{'app_id': 'com.hypercarrot.giantrush', 'cano...",2021-02-08T00:00:00Z,TapNation,TapNation,343356,26648,0.084141,582960,44324,0.082289
9,"[{'app_id': 'com.youmusic.magictiles', 'canoni...",2021-02-08T00:00:00Z,AMANOTES+PTE+LTD,AMANOTES PTE LTD,985876,70362,0.076855,469706,17232,0.038084


In [116]:
final_test = requests_Store_Summary('ios', '6014', 'US','week','2021-2-8','2021-2-14')
final_test

JSONDecodeError: Expecting value: line 1 column 1 (char 0)